In [1]:
import psi4 
import numpy as np

psi4.set_options({'basis':        'aug-cc-pvdz',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'e_convergence': 1e-14,
                  'd_convergence': 1e-8
                })
mol = psi4.geometry("""
C        0.000000    0.000000   -0.136477                                                                                                        
Li       0.000000    0.000000   -2.085527                                           
N        0.000000    0.000000    1.010778

units angstrom
""")
psi4.core.set_num_threads(3)

  Threads set to 3 by Python driver.


In [2]:
scf_e, scf_wfn = psi4.energy('SCF', return_wfn=True)
print(scf_e, mol.nuclear_repulsion_energy())

# MO coefficients and energies
Ca = np.array(scf_wfn.Ca_subset('AO','ALL'))
eps = np.array(scf_wfn.epsilon_a_subset('AO', 'ALL'))



Scratch directory: /home/dwave/psi4-scratch/files/

*** tstart() called on dwave-Precision-Tower-3620
*** at Wed Jan 19 16:52:37 2022

   => Loading Basis Set <=

    Name: AUG-CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry C          line   182 file /home/dwave/software/psi4/objdir/stage/share/psi4/basis/aug-cc-pvdz.gbs 
    atoms 2 entry LI         line    68 file /home/dwave/software/psi4/objdir/stage/share/psi4/basis/aug-cc-pvdz.gbs 
    atoms 3 entry N          line   218 file /home/dwave/software/psi4/objdir/stage/share/psi4/basis/aug-cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        3 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point gr

Module time:
	user time   =       2.07 seconds =       0.03 minutes
	system time =       0.04 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes
Total time:
	user time   =       2.07 seconds =       0.03 minutes
	system time =       0.04 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes


In [3]:
mints = psi4.core.MintsHelper(scf_wfn.basisset())
# The are 2-electron repulsion intergrals in AO basis
ao_erints = np.asarray(mints.ao_eri())

In [5]:
%%timeit
mo_erints_n8 = np.einsum('pqrs,pI,qJ,rK,sL->IJKL', ao_erints, Ca, Ca, Ca, Ca, optimize=True)


286 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
mo_erints = np.einsum('pqrs,pI->Iqrs', ao_erints, Ca)
mo_erints = np.einsum('Iqrs,qJ->IJrs', mo_erints, Ca)
mo_erints = np.einsum('IJrs,rK->IJKs', mo_erints, Ca)
mo_erints = np.einsum('IJKs,sL->IJKL', mo_erints, Ca)

7.35 s ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
mo_erints = np.einsum('pqrs,pI->Iqrs', ao_erints, Ca, optimize=True)
mo_erints = np.einsum('Iqrs,qJ->IJrs', mo_erints, Ca, optimize=True)
mo_erints = np.einsum('IJrs,rK->IJKs', mo_erints, Ca, optimize=True)
mo_erints = np.einsum('IJKs,sL->IJKL', mo_erints, Ca, optimize=True)

387 ms ± 5.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
size = Ca.shape[0]
mo_erints = np.dot(Ca.T, ao_erints.reshape(size, -1))
mo_erints = np.dot(mo_erints.reshape(-1, size), Ca)
mo_erints = mo_erints.reshape(size, size, size, size).transpose(1, 0, 3, 2)

mo_erints = np.dot(Ca.T, mo_erints.reshape(size, -1))
mo_erints = np.dot(mo_erints.reshape(-1, size), Ca)
mo_erints = mo_erints.reshape(size, size, size, size).transpose(1, 0, 3, 2)

192 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
np.allclose(moints, mo_erints)

NameError: name 'moints' is not defined

In [ ]:
%%timeit
mo_erints = np.einsum('pqrs,pI->Iqrs', ao_erints, Ca, optimize='greedy')
mo_erints = np.einsum('Iqrs,qJ->IJrs', mo_erints, Ca, optimize='greedy')
mo_erints = np.einsum('IJrs,rK->IJKs', mo_erints, Ca, optimize='greedy')
mo_erints = np.einsum('IJKs,sL->IJKL', mo_erints, Ca, optimize='greedy')

In [ ]:
path = np.einsum_path('pqrs,pI->Iqrs', ao_erints, Ca, optimize='optimal')[0]

In [ ]:
%%timeit
mo_erints = np.einsum('pqrs,pI->Iqrs', ao_erints, Ca, optimize=path)
mo_erints = np.einsum('Iqrs,qJ->IJrs', mo_erints, Ca, optimize=path)
mo_erints = np.einsum('IJrs,rK->IJKs', mo_erints, Ca, optimize=path)
mo_erints = np.einsum('IJKs,sL->IJKL', mo_erints, Ca, optimize=path)

In [ ]:
np.allclose(moints, mo_erints)